# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,katima mulilo,-17.5000,24.2667,29.00,5,2,2.89,NaN,1722245792
1,1,nizhneangarsk,55.7833,109.5500,15.16,82,63,1.34,RU,1722245792
2,2,polyarnyy,69.1989,33.4478,11.75,81,94,5.99,RU,1722245792
3,3,fort stockton,30.8940,-102.8793,24.31,68,0,0.89,US,1722245793
4,4,isafjordur,66.0755,-23.1240,11.02,87,100,1.03,IS,1722245793


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles ="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[
    (city_data_df["Max Temp"] < 27) 
    & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,fort stockton,30.8940,-102.8793,24.31,68,0,0.89,US,1722245793
44,44,kailua-kona,19.6406,-155.9956,26.98,82,0,2.57,US,1722245807
134,134,remire-montjoly,4.9167,-52.2667,25.02,88,0,0.00,GF,1722245836
183,183,gwanda,-20.9333,29.0000,25.55,7,0,2.94,ZW,1722245852
194,194,tolanaro,-25.0319,46.9987,26.04,57,0,2.06,MG,1722245856


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,fort stockton,US,30.8940,-102.8793,68,
44,kailua-kona,US,19.6406,-155.9956,82,
134,remire-montjoly,GF,4.9167,-52.2667,88,
183,gwanda,ZW,-20.9333,29.0000,7,
194,tolanaro,MG,-25.0319,46.9987,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort stockton - nearest hotel: No hotel found
kailua-kona - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
gwanda - nearest hotel: No hotel found
tolanaro - nearest hotel: No hotel found
bay city - nearest hotel: No hotel found
eastbourne - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
claiborne - nearest hotel: No hotel found
saint-francois - nearest hotel: No hotel found
maun - nearest hotel: No hotel found
andalusia - nearest hotel: No hotel found
nkowakowa - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
little river - nearest hotel: No hotel found
sao miguel do tapuio - nearest hotel: No hotel found
cartagena - nearest hotel: No hotel found
nieuw amsterdam - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
3,fort stockton,US,30.8940,-102.8793,68,No hotel found
44,kailua-kona,US,19.6406,-155.9956,82,No hotel found
134,remire-montjoly,GF,4.9167,-52.2667,88,No hotel found
183,gwanda,ZW,-20.9333,29.0000,7,No hotel found
194,tolanaro,MG,-25.0319,46.9987,57,No hotel found
217,bay city,US,43.5945,-83.8889,90,No hotel found
230,eastbourne,GB,50.7687,0.2845,65,No hotel found
231,vila velha,BR,-20.3297,-40.2925,90,No hotel found
247,holualoa,US,19.6228,-155.9522,73,No hotel found
279,claiborne,US,32.5160,-92.1918,90,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height=500,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)